In [1]:
# import csv

# orderers = []
# summary = []

# with open('D:\\me\\masters\\data\\google_books_1299.csv', encoding='utf-8') as file:
#     data = csv.reader(file)
#     for k in data:
#         print(k)

In [35]:
import csv 
import json
from bson.objectid import ObjectId

def csv_to_json(csvFilePath, jsonFilePath):
    jsonArray = []
    unusedColumns = ['','rating','voters','published_date','page_count']
      
    #read csv file
    with open(csvFilePath, encoding='utf-8') as csvf: 
        #load csv file data using csv library's dictionary reader
        csvReader = csv.DictReader(csvf)   

        #convert each csv row into python dict
        for row in csvReader: 
            row['price'] = round(float(row['price']) * 0.27, 2)
            row['currency'] = 'USD'
            row['generes'] = [element.strip() for element in list(row['generes'].split(","))]
            row['pageCount'] = int(row['page_count'])
            row['publishedDate'] = row['published_date']
            row['imageUrl'] = 'https://mastersimages.blob.core.windows.net/images/book.jpg'

            for column in unusedColumns:
                del row[column]
            
            #add this python dict to json array
            jsonArray.append(row)

    # print(jsonArray)

    result = list({v['title']:v for v in jsonArray}.values())
    print(len(result))
  
    # convert python jsonArray to JSON String and write to file
    with open(jsonFilePath, 'w', encoding='utf-8') as jsonf: 
        jsonString = json.dumps(result, indent=4)
        jsonf.write(jsonString)
          
csvFilePath = r'D:\\me\\masters\\data\\google_books_1299.csv'
jsonFilePath = r'D:\\me\\masters\\data\\books.json'
csv_to_json(csvFilePath, jsonFilePath)

246


In [2]:
import nest_asyncio
nest_asyncio.apply()

In [3]:
import motor.motor_asyncio
from config import *

client = motor.motor_asyncio.AsyncIOMotorClient(connectionString)
db = client.test

try:
    print(client.server_info())
except Exception:
    print("Unable to connect to the server.")

<Future pending cb=[_chain_future.<locals>._call_check_cancel() at C:\Users\User\AppData\Local\Programs\Python\Python310\lib\asyncio\futures.py:384]>


In [4]:
db = client['masters']
collection = db.books
collection

AsyncIOMotorCollection(Collection(Database(MongoClient(host=['masters-shard-00-01.cpq5u.mongodb.net:27017', 'masters-shard-00-00.cpq5u.mongodb.net:27017', 'masters-shard-00-02.cpq5u.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=False, retrywrites=True, w='majority', authsource='admin', replicaset='atlas-wrhudr-shard-0', ssl=True, driver=DriverInfo(name='Motor', version='2.5.1', platform='asyncio')), 'masters'), 'books'))

In [5]:
import asyncio

async def do_find_one(client):
    db = client['masters']
    collection = db.books
    # collection
    document = await collection.find_one({'price': {'$lt': 12}})
    print(document)

loop = asyncio.get_event_loop()
loop.run_until_complete(do_find_one(client))

{'_id': ObjectId('620ea6721feee707fc54937d'), 'title': 'Attack on Titan: Volume 13', 'author': 'Hajime Isayama', 'price': 11.69, 'currency': 'USD', 'description': "NO SAFE PLACE LEFT At great cost to the Garrison and the Survey Corps, Commander Erwin has managed to recover Eren from the Titans who tried to carry him off. But during the battle, Eren manifested yet another power he doesn't understand. As Eren and Krista find new enemies, the Survey Corps faces threats from both inside and outside the walls. And what will happen to Ymir, now that she has decided to make herself the Titans' prize?", 'publisher': 'Kodansha Comics', 'generes': ['none'], 'ISBN': '9781612626864', 'language': 'English', 'pageCount': 192, 'publishedDate': 'Jul 31, 2014', 'imageUrl': 'https://mastersimages.blob.core.windows.net/images/book.jpg'}


In [5]:
from bson.objectid import ObjectId

async def do_insert():
    result = await db.books.insert_many(
        [{ '_id': ObjectId(),'i': i} for i in range(3)])

loop = asyncio.get_event_loop()
loop.run_until_complete(do_insert())

In [34]:
def readJsonFile(fileName):
    with open(fileName, encoding='utf-8') as file: 
        # print(json.load(file))
        return json.load(file)

async def do_insert(dataset, fileName):
    data = readJsonFile(fileName)
    result = await db[dataset].insert_many(
        [ i for i in data])

loop = asyncio.get_event_loop()
loop.run_until_complete(do_insert('books', 'D:\\me\\masters\\data\\books.json'))

<coroutine object do_insert at 0x0000029750D527A0>

In [16]:
loop = asyncio.get_event_loop()
loop.run_until_complete(do_insert('users', 'D:\\me\\masters\\data\\users.json'))

In [9]:
import asyncio

async def do_delete_many():
    coll = db.rates
    n = await coll.count_documents({})
    print('%s documents before calling delete_many()' % n)
    result = await db.rates.delete_many({'i': {'$ne': ''}})
    print('%s documents after' % (await coll.count_documents({})))

loop = asyncio.get_event_loop()
loop.run_until_complete(do_delete_many())

10 documents before calling delete_many()
0 documents after


In [ ]:
import asyncio

async def do_delete_many():
    coll = db.rates
    n = await coll.count_documents({})
    print('%s documents before calling delete_many()' % n)
    result = await db.rates.delete_many({'i': {'$ne': ''}})
    print('%s documents after' % (await coll.count_documents({})))

loop = asyncio.get_event_loop()
loop.run_until_complete(do_delete_many())

In [7]:
async def do_find(dataset):
    result = []
    cursor = db[dataset].find({})
    for document in await cursor.to_list(length=1400):
        # print(document)
        result.append(document)
    return result

# loop = asyncio.get_event_loop()
# loop.run_until_complete(do_find())

In [10]:
import random
import datetime

loop = asyncio.get_event_loop()
books = loop.run_until_complete(do_find('books'))[0:10]
# books = do_find('books')

loop = asyncio.get_event_loop()
users = loop.run_until_complete(do_find('users'))[0:10]
# users = do_find('users')

rates = []

for i in range(10):
    dict = {}
    dict['bookId'] = books[i]['_id']
    dict_list = []

    for j in range(10):
        usersRandom = random.randint(0, 9)        

        rate = {}
        rate['userId'] = users[usersRandom]['_id']
        rate['rate'] = random.randint(1, 5)
        rate['comment'] = ''
        day_diff = random.randint(0, 3)
        yesterday = datetime.datetime.utcnow() - datetime.timedelta(days = day_diff)
        rate['createdAt'] = datetime.datetime(yesterday.year, yesterday.month, yesterday.day)

        existing = False
        
        # print(dict_list)
        for item in dict_list:
            # print(item)
            if(item['userId'] == rate['userId']):
                existing = True

        if(existing == False):
            dict_list.append(rate)

    dict['reviews'] = dict_list
    rates.append(dict)

# print(rates)

async def do_insert_data(data):
    result = await db.rates.insert_many(
        [ i for i in data])
    
loop = asyncio.get_event_loop()
loop.run_until_complete(do_insert_data(rates))

In [33]:
from dateutil import tz
today = datetime.datetime.utcnow().date()
start = datetime.datetime(today.year, today.month, today.day)
start

datetime.datetime(2022, 4, 27, 0, 0)

In [39]:
yesterday_datetime = datetime.datetime.utcnow() - datetime.timedelta(days = 1)
yesterday = datetime.datetime(yesterday_datetime.year, yesterday_datetime.month, yesterday_datetime.day)
today = yesterday + datetime.timedelta(days = 1)
today

datetime.datetime(2022, 4, 27, 0, 0)

In [12]:
date = Date()
type(date)

NameError: name 'Date' is not defined

In [39]:
orders = []

for i in range(300):
    booksAmount = random.randint(1, 5)
    usersRandom = random.randint(0, 119)

    dict = {}
    orderBooks = []
    
    dict['userId'] = users[usersRandom]['_id']

    for i in range(booksAmount):
        booksRandom = random.randint(0, 245)
        orderBooks.append(books[booksRandom]['_id'])

    dict['booksId'] = orderBooks

    orders.append(dict)
    # print(dict)


print(len(orders))

async def do_insert_data(data):
    result = await db.orders.insert_many(
        [ i for i in data])
    
loop = asyncio.get_event_loop()
loop.run_until_complete(do_insert_data(orders))

{'userId': ObjectId('620bb9fef23b1bc78052c651'), 'booksId': [ObjectId('620e9cb61feee707fc548d47'), ObjectId('620e9cb61feee707fc548d9e')]}
{'userId': ObjectId('620bb9fef23b1bc78052c5fb'), 'booksId': [ObjectId('620e9cb61feee707fc548d2a'), ObjectId('620e9cb61feee707fc548d59')]}
{'userId': ObjectId('620bb9fef23b1bc78052c63f'), 'booksId': [ObjectId('620e9cb61feee707fc548d7e'), ObjectId('620e9cb61feee707fc548d5a')]}
{'userId': ObjectId('620bb9fef23b1bc78052c5f2'), 'booksId': [ObjectId('620e9cb61feee707fc548d49'), ObjectId('620e9cb61feee707fc548d1b'), ObjectId('620e9cb61feee707fc548db3'), ObjectId('620e9cb61feee707fc548dd1'), ObjectId('620e9cb61feee707fc548dc0')]}
{'userId': ObjectId('620bb9fef23b1bc78052c618'), 'booksId': [ObjectId('620e9cb61feee707fc548d64')]}
{'userId': ObjectId('620bb9fef23b1bc78052c608'), 'booksId': [ObjectId('620e9cb61feee707fc548dc6'), ObjectId('620e9cb61feee707fc548dae')]}
{'userId': ObjectId('620bb9fef23b1bc78052c60f'), 'booksId': [ObjectId('620e9cb61feee707fc548d25'

In [1]:
import datetime
str = datetime.datetime.utcnow().strftime('%d.%m.%Y %H:%M:%S')
print(str)

datetime.datetime.strptime(str, '%d.%m.%Y %H:%M:%S')

23.02.2022 16:06:01


'23.02.2022 16:06:01'

In [15]:
datetime.datetime.utcnow() < (datetime.datetime.utcnow() - datetime.timedelta(days = 1))

False

In [13]:
(datetime.datetime.utcnow() - datetime.timedelta(days = 1)).strftime('%Y-%m-%d-%H:%M:%S')

'2022-02-21-18:09:51'

In [7]:
from bson.objectid import ObjectId
a = ObjectId()
a.__str__()

'62154d12e642272bf9189970'

In [5]:
a.equals('62154cbbe642272bf918996e')

AttributeError: 'ObjectId' object has no attribute 'equals'

In [5]:
import re

txt = datetime.datetime.utcnow().strftime('%d.%m.%Y %H:%M:%S')
x = re.search("^23.02.2022 ", txt)
x

<re.Match object; span=(0, 11), match='23.02.2022 '>

In [1]:
pip uninstall mongoengine

^C
Note: you may need to restart the kernel to use updated packages.


In [11]:
pip uninstall Flask-PyMongo

^C
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install flask_cors pymongo

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip flask_cors -v

Note: you may need to restart the kernel to use updated packages.


ERROR: unknown command "flask_cors"

